# Explore here

In [69]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB,GaussianNB, BernoulliNB
import warnings
from sklearn.exceptions import ConvergenceWarning
# Ignorar las advertencias de convergencia
warnings.filterwarnings("ignore", category=ConvergenceWarning)
data = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews.csv",sep=',')
data.to_csv('/workspaces/bayes-joel/data/raw/diabetes.csv',index=False)
data.head()

,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offli...,0
1,com.facebook.katana,"messenger issues ever since the last update, ...",0
2,com.facebook.katana,profile any time my wife or anybody has more ...,0
3,com.facebook.katana,the new features suck for those of us who don...,0
4,com.facebook.katana,forced reload on uploading pic on replying co...,0


In [70]:
#eliminamos la columna package_name

data.drop('package_name', axis=1,inplace=True)

In [71]:
#convertimos todas la spalabras en minusculas y eliminamos los espacios
data["review"] = data["review"].str.strip().str.lower()

In [72]:

X = data["review"]
y = data["polarity"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [73]:
#realizamoes el conteo de palabras 
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train).toarray()
X_test_vec = vectorizer.transform(X_test).toarray()

el que mejor resultado nos deberia dar es el MultinomialNB  por que el objetivo es binario y los predictores son numeros categoricos pero igualmente utilizaremos los 3 para compararlos 

In [74]:
#generamoslos modelo y luego me imprima el classification_report de cada uno de ellos 
modelos = [GaussianNB(), BernoulliNB(),MultinomialNB()]
modelos_entrenados = []
for modelo in modelos:
    modelo.fit(X_train_vec,y_train)
    y_pred = modelo.predict(X_test_vec)
    print(f'el modelo : {modelo} tiene las siguientes metricas:\n {classification_report(y_test, y_pred)}')
    modelos_entrenados.append((modelo))

el modelo : GaussianNB() tiene las siguientes metricas:
               precision    recall  f1-score   support

           0       0.84      0.89      0.86       126
           1       0.70      0.60      0.65        53

    accuracy                           0.80       179
   macro avg       0.77      0.75      0.76       179
weighted avg       0.80      0.80      0.80       179

el modelo : BernoulliNB() tiene las siguientes metricas:
               precision    recall  f1-score   support

           0       0.86      0.94      0.90       126
           1       0.81      0.64      0.72        53

    accuracy                           0.85       179
   macro avg       0.84      0.79      0.81       179
weighted avg       0.85      0.85      0.84       179

el modelo : MultinomialNB() tiene las siguientes metricas:
               precision    recall  f1-score   support

           0       0.85      0.95      0.90       126
           1       0.84      0.58      0.69        53

    acc

como podemos observar tanto en el BernoulliNB commo en el MultinomialNB() tenemos resultados bastante parecidos voy a proceder a buscar los mejores parametros para el  MultinomialNB() 

In [85]:
hyperparams = {
    "alpha": np.linspace(0.01, 10.0, 200),
    "fit_prior": [True, False]
}
random_search = RandomizedSearchCV(modelos_entrenados[2], hyperparams, n_iter = 50, scoring = "accuracy", cv = 5, random_state = 42)
random_search

RandomizedSearchCV(cv=5, estimator=MultinomialNB(), n_iter=50,
                   param_distributions={'alpha': array([ 0.01      ,  0.06020101,  0.11040201,  0.16060302,  0.21080402,
        0.26100503,  0.31120603,  0.36140704,  0.41160804,  0.46180905,
        0.51201005,  0.56221106,  0.61241206,  0.66261307,  0.71281407,
        0.76301508,  0.81321608,  0.86341709,  0.91361809,  0.9638191 ,
        1.0140201 ,  1.06422111,  1.11442211,  1.1646231...
        8.54417085,  8.59437186,  8.64457286,  8.69477387,  8.74497487,
        8.79517588,  8.84537688,  8.89557789,  8.94577889,  8.9959799 ,
        9.0461809 ,  9.09638191,  9.14658291,  9.19678392,  9.24698492,
        9.29718593,  9.34738693,  9.39758794,  9.44778894,  9.49798995,
        9.54819095,  9.59839196,  9.64859296,  9.69879397,  9.74899497,
        9.79919598,  9.84939698,  9.89959799,  9.94979899, 10.        ]),
                                        'fit_prior': [True, False]},
                   random_state=42, scoring='accuracy')

In [87]:
random_search.fit(X_train_vec, y_train) 

print(f"Best hyperparameters: {random_search.best_params_}")

Best hyperparameters: {'fit_prior': False, 'alpha': 0.6124120603015075}


In [88]:
model = MultinomialNB(alpha = 0.6124120603015075, fit_prior = False)

model.fit(X_train_vec, y_train)

y_predh = model.predict(X_test_vec)
print(classification_report(y_test, y_predh))


              precision    recall  f1-score   support

           0       0.88      0.90      0.89       126
           1       0.74      0.70      0.72        53

    accuracy                           0.84       179
   macro avg       0.81      0.80      0.80       179
weighted avg       0.84      0.84      0.84       179



en este caso tenemos que luego de ajustar los hiperparametros tenemso una precicion un poco mas alta que que antes para la clase 0 y tambien tenemos una sencibilidad mucho mas alta en la variable 1 que antes aunque es verdad que la precision en la clase 1 se reduce luego de ajustar el modelo considero que en general mejora la metricas 